#### 1. Setup and Dependencies

First, we'll ensure all necessary libraries are installed. Given your previous work with `lxml`, `PyMuPDF`, and `spaCy`, we'll include those for text extraction and potentially more advanced NLP preprocessing.

In [1]:
# 1.1. Install necessary libraries
# Use !pip install for notebook environment
# !pip install transformers trl accelerate bitsandbytes sentencepiece lxml PyMuPDF spacy peft
# !python -m spacy download en_core_web_sm # Download a small spaCy model

# 1.2. Import Libraries
import os
import re
import json
import pandas as pd
from dataclasses import dataclass, field, asdict
from typing import Set, List, Optional, Dict, Any

import fitz # PyMuPDF
from lxml import etree # For XML parsing
import spacy
import kagglehub

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils.quantization_config import BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from datasets import Dataset #, load_metric
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

from tqdm.auto import tqdm
import gc # For garbage collection

# For KaggleHub integration (assuming it's set up or models are downloaded)
# You might need to install kagglehub if you plan to use it directly for model download
# !pip install kagglehub

# 1.3. Configure CUDA for local GPU
if torch.cuda.is_available():
    print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
    torch.cuda.empty_cache() # Clear GPU memory
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")



CUDA is available! Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
# Define constants for file paths and model configurations
BASE_INPUT_DIR = './kaggle/input/make-data-count-finding-data-references'
BASE_OUTPUT_DIR = "./kaggle/working"

# Define directories for articles in train and test sets
TRAIN_DATA_DIR = os.path.join(BASE_INPUT_DIR, 'train')
TEST_DATA_DIR = os.path.join(BASE_INPUT_DIR, 'test')
TRAIN_LABELS_PATH = os.path.join(BASE_INPUT_DIR, 'train_labels.csv')

# Define the base model path
QWEN_BASE_MODEL_PATH = kagglehub.model_download("qwen-lm/qwen-3/transformers/0.6b")

# Output directory for the fine-tuned model and results
FINE_TUNED_MODEL_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, "results")
SAMPLE_SUBMISSION_PATH = os.path.join(BASE_OUTPUT_DIR, "submission.csv")

# Load spaCy model for sentence segmentation and potentially other NLP tasks
# python -m spacy download en_core_web_sm 
NLP_SPACY = spacy.load("en_core_web_sm")

In [59]:
# Information Extraction (IE) - Dataset Identification ---
NON_STD_UNICODE_DASHES = re.compile(r'[\u2010\u2011\u2012\u2013\u2014]')
NON_STD_UNICODE_TICKS = re.compile(r'[\u201c\u201d]')
def clean_text(text: str) -> str:
    """
    Clean the input text by removing non-standard unicode dashes and extra whitespace.
    
    Args:
        text (str): The text to clean.
        
    Returns:
        str: The cleaned text.
    """
    if not text:
        return ""
    # Replace all non-standard unicode dashes with '-'
    text = text.replace('\u200b', '').replace('-\n', '-').replace('_\n', '_').replace('/\n', '/').replace('dryad.\n', 'dryad.').replace('doi.\norg', 'doi.org')
    text = NON_STD_UNICODE_DASHES.sub('-', text)
    text = NON_STD_UNICODE_TICKS.sub("'", text)
    # Remove extra whitespace
    return re.sub(r'\s+', ' ', text).strip()

# Information Extraction (IE) - Dataset Identification
# Regex patterns for common dataset identifiers
DOI_PATTERN = r'\b10\.\d{4,5}\/[-._\/:A-Za-z0-9]+'
#DOI_PATTERN = r"(?:doi:|https?://(?:dx\.)?doi\.org/)(10\.\d{4,9}/[-._;()/:A-Z0-9]+)"
EPI_PATTERN = r'\bEPI[-_A-Z0-9]{2,}'
SAM_PATTERN = r'\bSAMN[0-9]{2,}'          # SAMN07159041
IPR_PATTERN = r'\bIPR[0-9]{2,}'
CHE_PATTERN = r'\bCHEMBL[0-9]{2,}'
PRJ_PATTERN = r'\bPRJ[A-Z0-9]{2,}'
E_G_PATTERN = r'\bE-[A-Z]{4}-[0-9]{2,}'   # E-GEOD-19722 or E-PROT-100
ENS_PATTERN = r'\bENS[A-Z]{4}[0-9]{2,}'
CVC_PATTERN = r'\bCVCL_[A-Z0-9]{2,}'
EMP_PATTERN = r'\bEMPIAR-[0-9]{2,}'
PXD_PATTERN = r'\bPXD[0-9]{2,}'
HPA_PATTERN = r'\bHPA[0-9]{2,}'
SRR_PATTERN = r'\bSRR[0-9]{2,}'
GSE_PATTERN = r'\b(GSE|GSM|GDS|GPL)\d{4,6}\b' # Example for GEO accession numbers (e.g., GSE12345, GSM12345)
GNB_PATTERN = r'\b[A-Z]{1,2}\d{5,6}\b' # GenBank accession numbers (e.g., AB123456, AF000001)
CAB_PATTERN = r'\bCAB[0-9]{2,}'
PDB_PATTERN = r"\bpdb\s*\d[A-Za-z0-9]{3}" # Example: pdb 5yfp

# Combine all patterns into a list
DATASET_ID_PATTERNS = [
    DOI_PATTERN,
    EPI_PATTERN,
    SAM_PATTERN,
    IPR_PATTERN,
    CHE_PATTERN,
    PRJ_PATTERN,
    E_G_PATTERN,
    ENS_PATTERN,
    CVC_PATTERN,
    EMP_PATTERN,
    PXD_PATTERN,
    HPA_PATTERN,
    SRR_PATTERN,
    GSE_PATTERN,
    GNB_PATTERN,
    CAB_PATTERN,
    PDB_PATTERN
]

# Compile all patterns for efficiency
COMPILED_DATASET_ID_REGEXES = [re.compile(p) for p in DATASET_ID_PATTERNS]

# Data related keywords to look for in the text
# These keywords help to ensure that the text is relevant to datasets
#DATA_RELATED_KEYWORDS = ['data release', 'data associated', 'data availability', 'data access', 'download', 'program data', 'the data', 'dataset', 'database', 'repository', 'data source', 'data access', 'archive', 'arch.', 'digital']
# DATA_RELATED_KEYWORDS = ['data ', 'dataset', 'database', 'download', 'repository', 'archive', 'arch.', 'digital']
#DATA_RELATED_KEYWORDS = ['data ', 'dataset', 'database']
DATA_RELATED_KEYWORDS = ['data release', 'data associated', 'data availability', 'data access', 'data source', 'program data', 'our data', 'the data', 'dataset', 'database',]

def is_text_data_related(text: str) -> bool:
    if not text:
        return False
    
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in DATA_RELATED_KEYWORDS)
    


#### 2. Data Classes


In [55]:
# 2.1. DatasetCitation Class
@dataclass
class DatasetCitation:
    dataset_id: str = ""
    citation_context: str = ""
    citation_type: Optional[str] = None # "Primary", "Secondary", or "Missing" - for ground truth during training

    def set_citation_context(self, context: str, check_data_related: bool = False):
        """Sets the citation context, cleaning it."""
        if context and (not check_data_related or not self.is_doi or is_text_data_related(context)):
            # Replace newlines with spaces, remove brackets, and normalize whitespace
            context = context.replace('\n', ' ').replace('[', '').replace(']', '')
            context = re.sub(r'\s+', ' ', context.strip())
            self.citation_context = context 

    def is_doi(self)-> bool:
        return self.dataset_id.startswith("10.")
    
    def has_dataset(self) -> bool:
        """Returns True if there are both dataset IDs and citation context."""
        return bool(self.dataset_id and self.citation_context.strip())

    def to_dict(self):
        return asdict(self)

# 2.2. ArticleData Class
@dataclass
class ArticleData:
    article_id: str = ""
    article_doi: str = ""
    title: str = ""
    author: str = ""
    abstract: str = ""
    dataset_citations: List[DatasetCitation] = field(default_factory=list)

    def __post_init__(self):
        # Custom initialization
        if self.article_id and not self.article_doi:
            # If article_id is provided but not article_doi, set article_doi
            self.article_doi = self.article_id.replace("_", "/").lower()

    def add_dataset_citation(self, dataset_citation: DatasetCitation):
        """Adds a DatasetCitation object to the article."""
        if dataset_citation.has_dataset() and dataset_citation.dataset_id != self.article_doi:
            self.dataset_citations.append(dataset_citation)
        
    def to_dict(self):
        d = asdict(self)
        # Convert list of DatasetCitation objects to their dict representation
        d["dataset_citations"] = [dc.to_dict() for dc in self.dataset_citations]
        return d

    def to_json(self):
        return json.dumps(self.to_dict(), separators=(',', ':'))

    def has_data(self) -> bool:
        """Returns True if there are any dataset citations."""
        return bool(self.dataset_citations)
    
@dataclass
class LlmTrainingData:
    article_id: str = ""
    article_doi: str = ""
    article_abstract: str = ""
    citation_context: str = ""
    dataset_id: str = ""
    label: str = ""

    def to_dict(self):
        return asdict(self)
    
    def to_json(self):
        return json.dumps(self.to_dict(), separators=(',', ':'))

@dataclass
class SubmissionData:
    article_id: str = ""
    dataset_id: str = ""
    type: str = ""
    context: str = ""

    def to_dict(self):
        return asdict(self)
    
    def to_json(self):
        return json.dumps(self.to_dict(), separators=(',', ':'))
    

#### 3. Data Loading and Initial Preprocessing

This section will cover how to load the raw competition data (full text articles and labels) and begin structuring it.

#### Load Labeled Training Data

In [12]:
def load_file_paths(dataset_type_dir: str) -> pd.DataFrame: 
    pdf_path = os.path.join(dataset_type_dir, 'PDF')
    xml_path = os.path.join(dataset_type_dir, 'XML')
    dataset_type = os.path.basename(dataset_type_dir)
    pdf_files = [f for f in os.listdir(pdf_path) if f.endswith('.pdf')]
    xml_files = [f for f in os.listdir(xml_path) if f.endswith('.xml')]
    df_pdf = pd.DataFrame({
        'article_id': [f.replace('.pdf', '') for f in pdf_files],
        'pdf_file_path': [os.path.join(pdf_path, f) for f in pdf_files]
    })
    df_xml = pd.DataFrame({
        'article_id': [f.replace('.xml', '') for f in xml_files],
        'xml_file_path': [os.path.join(xml_path, f) for f in xml_files]
    })
    merge_df = pd.merge(df_pdf, df_xml, on='article_id', how='outer', suffixes=('_pdf', '_xml'), validate="one_to_many")
    merge_df['dataset_type'] = dataset_type
    return merge_df

# Load the labeled training data CSV file
print(f"Loading labeled training data from: {TRAIN_LABELS_PATH}")
train_labels_df = pd.read_csv(TRAIN_LABELS_PATH)
print(f"Training labels shape: {train_labels_df.shape}")

# Group training data by article_id to get all datasets for each article
# This creates a dictionary where keys are article_ids and values are lists of dataset dicts
grouped_training_data = {}
for article_id, group_df in train_labels_df.groupby('article_id'):
    grouped_training_data[article_id] = group_df[['dataset_id', 'type']].to_dict('records')

# Example usage of grouped_training_data
print(f"Example grouped training data for article_id '10.1002_2017jc013030': {grouped_training_data['10.1002_2017jc013030']}")

# Just for testing, always set to the TEST_DATA_DIR
base_file_dir = TRAIN_DATA_DIR

# Load file paths for base directory
file_paths_df = load_file_paths(base_file_dir)
file_paths_df['xml_file_path'] = file_paths_df['xml_file_path'].fillna('')

# Merge the file paths with the grouped_training_data
file_paths_df['ground_truth_dataset_info'] = file_paths_df['article_id'].map(grouped_training_data)
file_paths_df['ground_truth_dataset_info'] = file_paths_df['ground_truth_dataset_info'].fillna('')

# Reduce the file paths DataFrame to only those with ground truth dataset info and get a sample
# This is to ensure we have a manageable dataset for training
file_paths_df = file_paths_df[file_paths_df['ground_truth_dataset_info'].astype(bool)]
file_paths_df = file_paths_df.reset_index(drop=True)
file_paths_df = file_paths_df.sample(frac=.5, random_state=42).reset_index(drop=True)  # Shuffle the DataFrame
print(f"Files paths shape: {file_paths_df.shape}")
display(file_paths_df.sample(3))

Loading labeled training data from: ./kaggle/input/make-data-count-finding-data-references\train_labels.csv
Training labels shape: (1028, 3)
Example grouped training data for article_id '10.1002_2017jc013030': [{'dataset_id': 'https://doi.org/10.17882/49388', 'type': 'Primary'}]
Files paths shape: (262, 5)


,article_id,pdf_file_path,xml_file_path,dataset_type,ground_truth_dataset_info
3,10.1111_2041-210x.12453,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,train,[{'dataset_id': 'https://doi.org/10.5061/dryad...
22,10.1186_s12920-020-00737-6,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,train,"[{'dataset_id': 'Missing', 'type': 'Missing'}]"
98,10.1186_s13750-020-0184-0,./kaggle/input/make-data-count-finding-data-re...,,train,"[{'dataset_id': 'Missing', 'type': 'Missing'}]"


#### Define File Extract Functions

In [ ]:
# 3.1. Helper function to extract text from various file types
def extract_text_from_file(filepath: str) -> str:
    """Extracts text from XML, PDF, or TXT files."""
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return ""
    
    print(f"Extracting text from file: {filepath}")
    if filepath.endswith(".xml"):
        parser = etree.XMLParser(resolve_entities=False, no_network=True)
        try:
            tree = etree.parse(filepath, parser)
            # A common way to get all text from an XML scientific article
            # This might need adjustment based on the specific XML schema
            return clean_text(" ".join(tree.xpath("//text()")).strip())
        except Exception as e:
            print(f"Error parsing XML {filepath}: {e}")
            return ""
    elif filepath.endswith(".pdf"):
        try:
            doc = fitz.open(filepath)
            text = ""
            for page in doc:
                text += page.get_textpage().extractTEXT()+"\n"
            return clean_text(text.strip())
        except Exception as e:
            print(f"Error parsing PDF {filepath}: {e}")
            return ""
    elif filepath.endswith(".txt"):
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read().strip()
    return ""

def extract_first_few_sentences(text: str, num_sentences: int = 5) -> str:
    """
    Extracts the first few sentences from the text.
    
    Args:
        text (str): The input text.
        num_sentences (int): The number of sentences to extract.
        
    Returns:
        str: The first few sentences from the text.
    """
    if not text:
        return ""
    
    doc = NLP_SPACY(text)
    sentences = list(doc.sents)
    
    # Join the first few sentences
    return " ".join([sent.text for sent in sentences[:num_sentences]]).strip()

def extract_article_data_from_text(full_text: str, article_id: str) -> ArticleData:
    """
    Extracts article data from the full text.
    
    Args:
        full_text (str): The full text of the article.
        article_id (str): The ID of the article.
        
    Returns:
        ArticleData: An instance of ArticleData with extracted information.
    """
    abstract_match = re.search(r"Abstract\s*(.*?)(?=\n\n|\Z)", full_text, re.IGNORECASE | re.DOTALL)
    abstract = abstract_match.group(1).strip() if abstract_match else "No Abstract"
    abstract = extract_first_few_sentences(abstract[:400], num_sentences=3)  # Extract first few sentences for the abstract

    return ArticleData(
        article_id=article_id,
        abstract=abstract
    )

# 4.2. Function to extract context around an ID
def extract_context_around_id(sentences, dataset_id: str, window_size_sentences: int = 3) -> str:
    """
    Extracts a window of sentences around a given dataset ID in the text.
    Uses spaCy for sentence segmentation.
    """
    if not sentences or not dataset_id or dataset_id == "Missing":
        return ""
        
    # Find all occurrences of the dataset_id (case-insensitive)
    matches = [(i, sent) for i, sent in enumerate(sentences) if dataset_id.lower() in sent.lower()]
    if not matches:
        return ""

    # For simplicity, take the context around the first match.
    # You might want to refine this to capture all relevant contexts or the most prominent one.
    first_match_idx = matches[0][0]
    
    start_idx = max(0, first_match_idx - window_size_sentences)
    end_idx = min(len(sentences), first_match_idx + 1)
    
    context_sentences = sentences[start_idx:end_idx]
    return " ".join(context_sentences)


def extract_training_data_for_llm(file_paths_df: pd.DataFrame) -> list[dict[str, str]]:
    """
    Extracts article data for training set with ground truth.
    
    Args:
        file_paths_df (pd.DataFrame): DataFrame containing file paths and ground truth info.
        
    Returns:
        Dict[str, ArticleData]: Dictionary mapping article IDs to ArticleData objects.
    """
    training_data_for_llm: list[dict[str, str]] = [] # This will be a list of LlmTrainingData for the LLM training dataset
    for i, row in tqdm(file_paths_df.iterrows(), total=len(file_paths_df)):
        article_id = row['article_id']
        filepath = row['pdf_file_path'] if row['pdf_file_path'] else row['xml_file_path']
        ground_truth_list = row['ground_truth_dataset_info'] if 'ground_truth_dataset_info' in row else []
        
        full_text = extract_text_from_file(filepath)
        article_data = extract_article_data_from_text(full_text, article_id)

        doc = NLP_SPACY(full_text)
        sentences = [sent.text for sent in doc.sents]

        if not ground_truth_list:
            print(f"No ground truth data found for article_id: {article_id}. Skipping this article.")
            continue
        for gt in ground_truth_list:
            dataset_id = gt['dataset_id'].replace("https://doi.org/", "").replace("doi:", "").strip()
            citation_type = gt.get('type', 'Primary')
            if dataset_id:
                # Convert to dict for LLM training data
                training_data_for_llm.append(
                    {
                        "article_id": article_data.article_id,
                        "article_doi": article_data.article_doi,
                        "article_abstract": article_data.abstract,
                        "citation_context": extract_context_around_id(sentences, dataset_id),
                        "dataset_id": dataset_id,
                        "label": citation_type
                    }
                )

    print(f"Loaded training data for {len(training_data_for_llm)} articles.")
    return training_data_for_llm





In [14]:
# For testing, let's extract training data for a specific article
sample_file_paths_df = file_paths_df.loc[file_paths_df['article_id'] == '10.1002_esp.5058']
sample_file_paths_df

,article_id,pdf_file_path,xml_file_path,dataset_type,ground_truth_dataset_info
154,10.1002_esp.5058,./kaggle/input/make-data-count-finding-data-re...,,train,[{'dataset_id': 'https://doi.org/10.5061/dryad...


#### 4. Advanced Preprocessing: Extracting Dataset Mentions and Context (Training)

Use regex to find the given dataset IDs from the training_labels and then use spaCy to extract surrounding sentences as context.


In [ ]:
# 4.3. Populate ArticleData with DatasetCitation objects and ground truth
testing_data_for_llm = extract_training_data_for_llm(file_paths_df)
print(f"Prepared {len(testing_data_for_llm)} training examples for the LLM.")

# Convert the list of LlmTrainingData to a DataFrame and save it
training_data_for_llm_df = pd.DataFrame(testing_data_for_llm)
training_data_for_llm_df.to_csv(os.path.join(BASE_OUTPUT_DIR, "training_data_for_llm.csv"), index=False)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_list(testing_data_for_llm)
train_dataset = train_dataset.shuffle(seed=42) # Shuffle for good measure

# Split into train/validation
train_test_split = train_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']
print(f"Training set size: {len(train_dataset)} examples")
print(f"Validation set size: {len(eval_dataset)} examples")

  0%|          | 0/262 [00:00<?, ?it/s]

Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.7717_peerj.12422.pdf
Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.1371_journal.pone.0198382.pdf
Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.1002_chem.202000235.pdf
Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.1111_2041-210x.12453.pdf
Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.1038_s41597-019-0101-y.pdf
Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.1186_s12974-020-01860-y.pdf
Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.7717_peerj.13193.pdf
Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\train\PDF\10.3390_s23177333.pdf
Extracting text from file: ./kaggle/input/mak

In [ ]:
# Save off the datasets to CSV for later use
train_dataset.to_csv(os.path.join(BASE_OUTPUT_DIR, "train_dataset.csv"), index=False)
eval_dataset.to_csv(os.path.join(BASE_OUTPUT_DIR, "eval_dataset.csv"), index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

53426

#### 5. Model Selection and Configuration

We'll use a Qwen model.

In [17]:
# 5.1. Choose a Model from KaggleHub
# Example: Qwen/Qwen1.5-0.5B-Chat (or 1.8B-Chat if 0.5B is too small/performs poorly)
# You can find these on KaggleHub or Hugging Face Hub.
model_name = QWEN_BASE_MODEL_PATH

# 5.2. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Qwen uses EOS for padding

# 5.3. Load Model with Quantization (4-bit)
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 # Or torch.float16 if bfloat16 is not supported by your GPU
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16, # Match compute_dtype
    device_map="auto", # Automatically maps model to available devices
    trust_remote_code=True # Required for some models like Qwen
)

# Prepare model for k-bit training (LoRA compatible)
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

print(f"Model {model_name} loaded with 4-bit quantization.")

Model C:\Users\jim\.cache\kagglehub\models\qwen-lm\qwen-3\transformers\0.6b\1 loaded with 4-bit quantization.


#### 6. Dataset Preparation for Training

Format the extracted data into instruction-tuning prompts using the ChatML format, which Qwen models are trained on.

In [18]:
# 6.1. Define the formatting function for ChatML (Corrected for trl 0.19.1)
def format_example(example):
    messages = [
        {"role": "system", "content": "You are an expert assistant for classifying research data citations. /no_think"},
        {"role": "user", "content": (
            f"Given the following article context and a specific data citation, classify if the data was generated as 'Primary' (newly generated for this study), 'Secondary' (reused from existing records), or 'Missing' (no data citation context given).\n\n"
            f"Article DOI: {example['article_doi']}\n"
            f"Article Abstract: {example['article_abstract']}\n" 
            f"Data Citation Context: {example['citation_context']}\n"
            f"Dataset ID: {example['dataset_id']}\n\n"
            f"Classification:"
        )}
    ]
    # The target output for the model is just "Primary" or "Secondary"
    messages.append({"role": "assistant", "content": example['label']})
    
    # Apply chat template and return the string directly
    # <--- IMPORTANT CHANGE: Directly return the string, not a dictionary
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False, enable_thinking=False)

# Apply the formatting to the dataset
# IMPORTANT: When formatting_func returns a string directly, you typically don't
# need to call .map() on the dataset beforehand if SFTTrainer handles it internally.
# However, if you want to inspect the formatted text, you can still do this:
# formatted_train_dataset = train_dataset.map(format_example)
# But for SFTTrainer, you pass the original `train_dataset` and the `formatting_func`
# and `dataset_text_field` (which will be ignored if formatting_func is used to generate the text).

# Print an example to verify (you'll need to call format_example directly for this)
print("\nExample of formatted training data (string output):")
# You can't directly print from formatted_train_dataset if you don't map it first.
# Let's print by calling the function on a sample:
if len(train_dataset) > 0:
    sample_formatted_text = format_example(train_dataset[0])
    print(sample_formatted_text)
else:
    print("No training data to display example.")


Example of formatted training data (string output):
<|im_start|>system
You are an expert assistant for classifying research data citations. /no_think<|im_end|>
<|im_start|>user
Given the following article context and a specific data citation, classify if the data was generated as 'Primary' (newly generated for this study), 'Secondary' (reused from existing records), or 'Missing' (no data citation context given).

Article DOI: 10.1111/cas.12935
Article Abstract: No Abstract
Data Citation Context: The difference was more significant in ER-negative BCs (P = 0.0276 for ER-negative vs healthy control; P = 0.2277 for ER-positive vs healthy control). The area under the curve value was 0.604 for all BC patients versus healthy Table 1. Twelve proteins selected as marker candidates for breast cancer Accession no. Protein name Gene name MS ⁄ MS spectral count HPA database MCF-7 MDA-MB-231 SK-BR-3 Hs578T HPA Ab† Average IHC score Percent of location Normal Breast cancer n ⁄ c ⁄ m c ⁄ m n Q99538 L

In [20]:
# ---------------------------------------------------------
# This version uses the evaluation dataset in the SFTTrainer
# ---------------------------------------------------------

# 7.1. Configure LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear", # Adjust based on model architecture if needed
)

# 7.2. Configure Training Arguments (now using SFTConfig)
training_args = SFTConfig(
    output_dir=FINE_TUNED_MODEL_OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    optim="paged_adamw_8bit",
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="none",
    disable_tqdm=False,
    remove_unused_columns=False,
    label_names=[],
    
    # SFTTrainer-specific parameters moved into SFTConfig
    max_seq_length=256,
    packing=False,
    dataset_text_field="text",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False},

    # --- NEW: Evaluation Parameters ---
    eval_strategy="steps", # Evaluate every 'eval_steps'
    eval_steps=500,              # How often to run evaluation (e.g., every 500 steps)
                                 # You can also use "epoch" for evaluation_strategy
    save_strategy="steps",       # How often to save checkpoints
    save_total_limit=1,          # Only keep the best model checkpoint
    load_best_model_at_end=True, # Load the model with the best validation metric at the end of training
    metric_for_best_model="eval_loss", # Metric to monitor for best model (default for CLM)
    greater_is_better=False,     # For loss, lower is better
)

# 7.3. Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, # <--- Pass the evaluation dataset here
    peft_config=peft_config,
    args=training_args,
    formatting_func=format_example
)


Applying formatting function to train dataset:   0%|          | 0/515 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/515 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/515 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/515 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/58 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/58 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/58 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/58 [00:00<?, ? examples/s]

In [ ]:
# ---------------------------------------------------------
# This version works but does NOT use the evaluation dataset
# ---------------------------------------------------------

# # 7.1. Configure LoRA
# peft_config = LoraConfig(
#     lora_alpha=16,
#     lora_dropout=0.1,
#     r=64,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules="all-linear", # Adjust based on model architecture if needed
# )

# # 7.2. Configure Training Arguments (now using SFTConfig)
# # SFTConfig combines TrainingArguments with SFTTrainer-specific parameters
# training_args = SFTConfig( # <--- IMPORTANT CHANGE: Use SFTConfig instead of TrainingArguments
#     output_dir=FINE_TUNED_MODEL_OUTPUT_DIR,
#     per_device_train_batch_size=1, # Adjust based on your GPU memory
#     gradient_accumulation_steps=16,
#     learning_rate=2e-4,
#     num_train_epochs=3,
#     logging_steps=10,
#     save_steps=500,
#     optim="paged_adamw_8bit",
#     fp16=True,  # <--- CHANGED: Try fp16 for broader compatibility and memory
#     bf16=False, # <--- CHANGED: Disable bf16 if fp16 is used
#     max_grad_norm=0.3,
#     warmup_ratio=0.03,
#     lr_scheduler_type="constant",
#     report_to="none",
#     disable_tqdm=False,
#     remove_unused_columns=False, # Keep columns for formatting
#     label_names=[], # Explicitly tell Trainer not to look for label columns in the dataset
#     # Additional SFT-specific parameters    
#     max_seq_length=512, # Max input sequence length (adjust based on context size)
#     packing=False, # Set to True for more efficient training if your data is short
#     dataset_text_field="text", # The name of the column in your dataset containing the text
#     # <--- NEW: Enable gradient checkpointing
#     gradient_checkpointing=True,
#     # This line is important for gradient checkpointing with PeftModel
#     # It tells the model to use the Peft (LoRA) layers for checkpointing
#     gradient_checkpointing_kwargs={'use_reentrant':False} # Recommended for newer PyTorch/Accelerate
# )

# # 7.3. Initialize SFTTrainer (Corrected for trl 0.19.1)
# trainer = SFTTrainer(
#     model=model,
#     processing_class=tokenizer, 
#     train_dataset=train_dataset,
#     peft_config=peft_config,
#     args=training_args, # This is now an SFTConfig object
#     formatting_func=format_example # This remains a direct argument to SFTTrainer
# )


Applying formatting function to train dataset:   0%|          | 0/27 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/27 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/27 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/27 [00:00<?, ? examples/s]

In [21]:

# 7.4. Start Training
print("\nStarting model training...")
trainer.train()
print("Training complete!")

# Save the fine-tuned model (LoRA adapters)
trainer.save_model(os.path.join(FINE_TUNED_MODEL_OUTPUT_DIR, "final_model"))
print(f"Fine-tuned model saved to {os.path.join(FINE_TUNED_MODEL_OUTPUT_DIR, 'final_model')}")


Starting model training...


Step,Training Loss,Validation Loss


Training complete!
Fine-tuned model saved to ./kaggle/working\results\final_model


In [ ]:
# --- Explicit GPU Memory Cleanup ---
print("\nInitiating GPU memory cleanup...")

# 1. Explicitly delete large objects that consume GPU memory
#    This removes references, allowing Python's garbage collector to act.
if 'trainer' in locals() and trainer is not None:
    del trainer
if 'model' in locals() and model is not None:
    del model
if 'tokenizer' in locals() and tokenizer is not None:
    del tokenizer
# If you had other large tensors or datasets explicitly moved to GPU,
# you would delete them here too. For Hugging Face datasets, they are usually
# on CPU unless you manually call .to('cuda').

# 2. Force Python's garbage collection
#    This helps ensure that deleted objects are immediately cleaned up.
gc.collect()

# 3. Clear PyTorch's CUDA memory cache
#    This tells PyTorch to release any cached memory back to the OS/driver.
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("GPU memory cleanup complete. Please check nvidia-smi to confirm.")


Initiating GPU memory cleanup...
GPU memory cleanup complete. Please check nvidia-smi to confirm.


#### 8. Inference and Evaluation

After training, load the best model (or the final one) and apply it to the test data.

In [22]:
# 8.1. Load the Trained Model (or merge LoRA adapters for full model)
# If you saved LoRA adapters, you'll need to load the base model and then the adapters.
# For inference, it's often easier to merge them.
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=nf4_config, # Use the same config as training
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     trust_remote_code=True
# )
# model = PeftModel.from_pretrained(model, os.path.join(FINE_TUNED_MODEL_OUTPUT_DIR, "final_model"))
# model = model.merge_and_unload() # Merge LoRA adapters into the base model

# For simplicity, if you just want to test the last saved checkpoint:
# You can also load the model directly from the checkpoint if it's a full save
# model = AutoModelForCausalLM.from_pretrained(os.path.join(FINE_TUNED_MODEL_OUTPUT_DIR, "final_model"), device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(os.path.join(FINE_TUNED_MODEL_OUTPUT_DIR, "final_model"))

# If you want to load the base model and then the adapters for inference:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
model = PeftModel.from_pretrained(model, os.path.join(FINE_TUNED_MODEL_OUTPUT_DIR, "final_model"))
model.eval() # Set to evaluation mode

print("Model loaded for inference.")


Model loaded for inference.


In [46]:
# For testing, always set to the TEST_DATA_DIR
base_file_dir = TEST_DATA_DIR

# Load file paths for base directory
test_file_paths_df = load_file_paths(base_file_dir)
test_file_paths_df['xml_file_path'] = test_file_paths_df['xml_file_path'].fillna('')

print(f"Files paths shape: {test_file_paths_df.shape}")
display(test_file_paths_df.sample(3))

Files paths shape: (30, 4)


,article_id,pdf_file_path,xml_file_path,dataset_type
0,10.1002_2017jc013030,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test
5,10.1002_chem.201903120,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test
20,10.1002_ecs2.1280,./kaggle/input/make-data-count-finding-data-re...,,test


In [56]:
def find_potential_dataset_ids(text: str) -> list[str]:
    """
    Finds potential dataset IDs in the given text using predefined regex patterns.
    
    Args:
        text (str): The input text to search for dataset IDs.
        
    Returns:
        Set[str]: A set of unique dataset IDs found in the text.
    """
    dataset_ids = set()
    for regex in COMPILED_DATASET_ID_REGEXES:
        for match in re.finditer(regex, text):
            dataset_id = match.group(0)
            dataset_ids.add(dataset_id)
    return list(dataset_ids)

def invoke_model_for_inference(tokenizer, article_data: ArticleData) -> list[SubmissionData]:
    submission_data_list = []
    article_id = article_data.article_id
    dataset_citations = article_data.dataset_citations
    if not dataset_citations:
        submission_data_list.append(SubmissionData(article_id, dataset_id="Missing", type="Missing"))
        return submission_data_list

    print(f"Found {len(dataset_citations)} citations for {article_id}")
    for dc in dataset_citations:
        # Create the prompt for inference
        messages = [
            {"role": "system", "content": "You are an expert assistant for classifying research data citations. /no_think"},
            {"role": "user", "content": (
                f"Given the following article context and a specific data citation, classify if the data was generated as 'Primary' (newly generated for this study) or 'Secondary' (reused from existing records).\n\n"
                f"Article Title: {article_data.title}\n"
                f"Article Abstract: {article_data.abstract}\n"
                f"Data Citation Context: {dc.citation_context}\n"
                f"Dataset ID: {dc.dataset_id}\n\n"
                f"Classification:"
            )}
        ]

        # --- CHANGE STARTS HERE ---
        # Tokenize and get both input_ids and attention_mask
        input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

        with torch.no_grad():
            output = model.generate(
                **inputs, # <--- Pass the entire dictionary (includes input_ids and attention_mask)
                max_new_tokens=10, # Expecting "Primary" or "Secondary"
                do_sample=False, # Use greedy decoding as per your preference
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        # --- CHANGE ENDS HERE ---        

        generated_text = tokenizer.decode(output[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip() # Use inputs['input_ids']
        # print(f"LLM Resp: {generated_text}")        
        
        # Post-process the generated text to get the classification
        predicted_type = "Missing"
        if "Primary" in generated_text:
            predicted_type = "Primary"
        elif "Secondary" in generated_text:
            predicted_type = "Secondary"
        
        submission_data_list.append(SubmissionData(article_id, dataset_id=dc.dataset_id, type=predicted_type, context=dc.citation_context))

    return submission_data_list

def extract_article_data_for_inference(article_id: str, filepath: str) -> ArticleData:
    full_text = extract_text_from_file(filepath)
    article_data = extract_article_data_from_text(full_text, article_id)
    potential_dataset_ids = find_potential_dataset_ids(full_text)
    if potential_dataset_ids:
        doc = NLP_SPACY(full_text)
        sentences = [sent.text for sent in doc.sents]

        # Populate article_data with potentially valid dataset_citations
        # the set_citation_context and add_dataset_citation methods do all of the appropriate filtering
        for dataset_id in potential_dataset_ids:
            citation = DatasetCitation(dataset_id=dataset_id)
            context = extract_context_around_id(sentences, dataset_id)
            citation.set_citation_context(context, check_data_related=True)
            article_data.add_dataset_citation(citation)

    return article_data

def process_test_articles(tokenizer, file_paths_df: pd.DataFrame) -> list[SubmissionData]:
    """
    Extracts article data for testing set without ground truth.
    
    Args:
        file_paths_df (pd.DataFrame): DataFrame containing file paths and ground truth info.
        
    Returns:
        Dict[str, ArticleData]: Dictionary mapping article IDs to ArticleData objects.
    """
    submission_data_list = []
    for i, row in tqdm(file_paths_df.iterrows(), total=len(file_paths_df)):
        article_id = row['article_id']
        filepath = row['pdf_file_path'] if row['pdf_file_path'] else row['xml_file_path']
        
        # Extract article data
        article_data = extract_article_data_for_inference(article_id, filepath)

        # Invoke the model with the collected article_data
        submission_data_list.extend(invoke_model_for_inference(tokenizer, article_data))

    print(f"Processed testing data for {len(submission_data_list)} article and dataset_id combos.")
    return submission_data_list

In [41]:
text = "adff 10.1234/dryad.as2345 accession numbers GSE1 GSE37569, GSE45042 , GSE28166  (pdb 5yfp) 10.25386/genetics.11365982 "
rsp1 = find_potential_dataset_ids(text)
display(rsp1)


['GSE45042',
 '10.1234/dryad.as2345',
 'GSE28166',
 'GSE37569',
 '10.25386/genetics.11365982',
 'pdb 5yfp']

In [47]:
sample_test_file_paths_df = test_file_paths_df.sample(2, random_state=42)
sample_test_file_paths_df

,article_id,pdf_file_path,xml_file_path,dataset_type
27,10.1002_mp.14424,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test
15,10.1002_ece3.6144,./kaggle/input/make-data-count-finding-data-re...,./kaggle/input/make-data-count-finding-data-re...,test


In [60]:
sample_sub = process_test_articles(tokenizer, test_file_paths_df)
display(sample_sub)

  0%|          | 0/30 [00:00<?, ?it/s]

Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1002_2017jc013030.pdf


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Found 3 citations for 10.1002_2017jc013030


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Found 4 citations for 10.1002_nafm.10870


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Extracting text from file: ./kaggle/input/make-data-count-finding-data-references\test\PDF\10.1007_jhep07(2018)134.pdf
Processed testing data for 46 article and dataset_id combos.


[SubmissionData(article_id='10.1002_2017jc013030', dataset_id='10.17882/49388', type='Primary', context='Xing, X. (2017). A global database of vertical profiles derived from Biogeochemical Argo float measurements for biogeochemical and bio-optical applications. SEANOE. https://doi.org/10.17882/49388'),
 SubmissionData(article_id='10.1002_2017jc013030', dataset_id='10.17882/47142', type='Secondary', context='Approches Num�eriques); Pierre-Marie Poulain (National Institute of Oceanography and Experimental Geophysics, Italy; Argo-Italy); Sabrina Speich (Laboratoire de M�et�eorologie Dynamique, France; LEFE-GMMC); Virginie Thierry (Ifremer, France; LEFE-GMMC); Pascal Conan (Observatoire Oc�eanologique de Banyuls sur mer, France; LEFE-GMMC); Laurent Coppola (Laboratoire d’Oc�eanographie de Villefranche, France; LEFE-GMMC); Anne Petrenko (Mediterranean Institute of Oceanography, France; LEFE-GMMC); and Jean-Baptiste Sall�ee (Laboratoire d’Oc�eanographie et du Climat, France; LEFE-GMMC). Coll

In [ ]:

# 8.2. Preprocess Test Data (similar to training data)
test_articles_data: Dict[str, ArticleData] = {}
# Assuming test data structure is similar to train data (full text files)
for article_file in os.listdir(TEST_DATA_DIR):
    article_id = os.path.splitext(article_file)[0]
    filepath = os.path.join(TEST_DATA_DIR, article_file)
    
    full_text = extract_text_from_file(filepath)
    
    # Extract title, author, abstract (same as training)
    title_match = re.search(r"Title:\s*(.*)", full_text, re.IGNORECASE)
    title = title_match.group(1).strip() if title_match else "Unknown Title"
    author_match = re.search(r"Author(?:s)?:\s*(.*)", full_text, re.IGNORECASE)
    author = author_match.group(1).strip() if author_match else "Unknown Author"
    abstract_match = re.search(r"Abstract\s*(.*?)(?=\n\n|\Z)", full_text, re.IGNORECASE | re.DOTALL)
    abstract = abstract_match.group(1).strip() if abstract_match else "No Abstract"

    article_data = ArticleData(
        article_id=article_id,
        title=title,
        author=author,
        abstract=abstract
    )
    # For test data, we need to find *all* potential dataset IDs, not just ground truth
    # This is the "finding datasets" part of your goal.
    
    # Use regex to find all potential dataset IDs in the full text
    found_dataset_mentions = []
    for pattern in ALL_ID_PATTERNS:
        for match in re.finditer(pattern, full_text, re.IGNORECASE):
            dataset_id = match.group(1) if pattern == DOI_PATTERN else match.group(0)
            span_text = match.group(0) # The full matched text
            
            context = extract_context_around_id(full_text, span_text, window_size_sentences=3)
            
            if context:
                dc = DatasetCitation()
                dc.add_dataset_id(dataset_id)
                dc.set_citation_context(context)
                found_dataset_mentions.append(dc)
                
    article_data.dataset_citations = found_dataset_mentions # Assign found mentions
    test_articles_data[article_id] = article_data

print(f"Prepared {len(test_articles_data)} test articles for inference.")

# 8.3. Generate Predictions
predictions = []
true_labels = [] # Only if you have a test_labels.json for evaluation

for article_id, article_data in test_articles_data.items():
    for dc in article_data.dataset_citations:
        # Create the prompt for inference
        messages = [
            {"role": "system", "content": "You are an expert assistant for classifying research data citations."},
            {"role": "user", "content": (
                f"Given the following article context and a specific data citation, classify if the data was generated as 'Primary' (newly generated for this study) or 'Secondary' (reused from existing records).\n\n"
                f"Article Title: {article_data.title}\n"
                f"Article Abstract: {article_data.abstract}\n"
                f"Data Citation Context: {dc.citation_context}\n"
                f"Dataset ID: {list(dc.dataset_ids)[0]}\n\n" # Assuming one ID per citation
                f"Classification:"
            )}
        ]
        
        input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids.to(device)

        with torch.no_grad():
            output = model.generate(
                input_ids,
                max_new_tokens=10, # Expecting "Primary" or "Secondary"
                do_sample=False, # Use greedy decoding as per your preference
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        generated_text = tokenizer.decode(output[0][input_ids.shape[1]:], skip_special_tokens=True).strip()
        
        # Post-process the generated text to get the classification
        predicted_type = "Unknown"
        if "Primary" in generated_text:
            predicted_type = "Primary"
        elif "Secondary" in generated_text:
            predicted_type = "Secondary"
        
        predictions.append({
            "article_id": article_id,
            "dataset_id": list(dc.dataset_ids)[0],
            "predicted_type": predicted_type
        })

        # If you have test labels, you can collect true_labels here for evaluation
        # For Kaggle, you'll typically submit predictions without knowing test labels.

print(f"Generated {len(predictions)} predictions.")

# 8.4. Evaluation (if test labels are available)
# If you have a separate test_labels.json for local evaluation:
# test_labels = load_labels(TEST_LABELS_PATH) # Load test labels
#
# # Match predictions to true labels and calculate metrics
# # This part requires careful matching of dataset_id within article_id
# # and might involve fuzzy matching for context if exact span isn't available.
# # For simplicity, assuming exact match on article_id and dataset_id.
#
# y_true = []
# y_pred = []
#
# for pred_entry in predictions:
#     article_id = pred_entry["article_id"]
#     dataset_id = pred_entry["dataset_id"]
#     predicted_type = pred_entry["predicted_type"]
#
#     # Find the true label for this specific dataset_id in this article
#     found_true_label = False
#     if article_id in test_labels:
#         for gt_info in test_labels[article_id]:
#             if gt_info["dataset_id"] == dataset_id: # Exact match on ID
#                 y_true.append(gt_info["citation_type"])
#                 y_pred.append(predicted_type)
#                 found_true_label = True
#                 break
#     if not found_true_label:
#         # Handle cases where a predicted ID might not be in ground truth
#         # or where the ID extraction was imperfect.
#         # For competition, this means your ID extraction needs to be precise.
#         pass
#
# if y_true and y_pred:
#     from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
#     # Map "Primary" to 1, "Secondary" to 0 for sklearn metrics
#     label_map = {"Primary": 1, "Secondary": 0}
#     y_true_mapped = [label_map.get(l, -1) for l in y_true]
#     y_pred_mapped = [label_map.get(l, -1) for l in y_pred]
#
#     # Filter out -1 if there were unknown labels
#     valid_indices = [i for i, val in enumerate(y_true_mapped) if val != -1 and y_pred_mapped[i] != -1]
#     y_true_mapped = [y_true_mapped[i] for i in valid_indices]
#     y_pred_mapped = [y_pred_mapped[i] for i in valid_indices]
#
#     if y_true_mapped:
#         print("\nEvaluation Results:")
#         print(f"Accuracy: {accuracy_score(y_true_mapped, y_pred_mapped):.4f}")
#         print(f"F1 Score (weighted): {f1_score(y_true_mapped, y_pred_mapped, average='weighted'):.4f}")
#         print(f"Precision (weighted): {precision_score(y_true_mapped, y_pred_mapped, average='weighted'):.4f}")
#         print(f"Recall (weighted): {recall_score(y_true_mapped, y_pred_mapped, average='weighted'):.4f}")
#     else:
#         print("No matching true labels found for evaluation.")
# else:
#     print("Not enough data to perform evaluation.")

#### 9. Submission File Generation (Kaggle Specific)

Finally, format your predictions into the required `submission.csv` file.

In [ ]:
# 9.1. Create Submission DataFrame

submission_df = pd.DataFrame(predictions)
# Rename columns to match Kaggle's expected format (e.g., 'id', 'class_label')
# This will depend on the exact submission format specified by Kaggle.
# Example:
# submission_df = submission_df.rename(columns={"article_id": "Id", "dataset_id": "DatasetId", "predicted_type": "Type"})
# submission_df["Id"] = submission_df["Id"] + "_" + submission_df["DatasetId"] # If Id is a combination

# Assuming the submission format is a list of dictionaries with 'article_id', 'dataset_id', 'citation_type'
# You might need to adjust this based on the exact competition requirements.
# For example, if it expects a single ID column like "article_id_dataset_id"
final_submission_data = []
for pred in predictions:
    final_submission_data.append({
        "Id": f"{pred['article_id']}_{pred['dataset_id']}", # Example: combine IDs
        "Type": pred['predicted_type']
    })

final_submission_df = pd.DataFrame(final_submission_data)
final_submission_df.to_csv("submission.csv", index=False)

print("Submission file 'submission.csv' created successfully!")

In [ ]:
def f1_score(tp, fp, fn):
    return 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 0.0
    
    
# if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
pred_df = submission_df.copy()
label_df = pd.read_csv("./kaggle/input/make-data-count-finding-data-references/sample_submission.csv")
label_df = label_df[label_df['type'] != 'Missing'].reset_index(drop=True)

hits_df = label_df.merge(pred_df, on=["article_id", "dataset_id", "type"])

tp = hits_df.shape[0]
fp = pred_df.shape[0] - tp
fn = label_df.shape[0] - tp
